# 第13章 平稳时间序列

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

data = pd.read_stata('../2_Data/Data-2e/gdp_china.dta')
data.set_index('year', inplace=True)
data['lny'] = np.log(data['y'])
data['dlny']  = data['lny'].diff().dropna()
data['dy'] = (data['y'] - data['y'].shift(1))/data['y'].shift(1)


class Dataset:
    def __init__(self, data):
        self.original = data
        self.dif = self.dif_y()
        self.lny = self.ln_y()
        self.dlny = self.dln_y()
        self.dy = self.d_y()



## 13.1 时间序列的自相关

自相关图（ACF）是描述时间序列自相关性的图形。它显示了时间序列的相关性随时间的变化情况。

ACF的计算方法：

1. 计算时间序列的差分值（即滞后1阶）
2. 计算差分值的自相关函数

ACF的计算公式：

$$
\rho_k = \frac{\sum_{t=1}^{T-k}(y_t-\bar{y})(y_{t+k}-\bar{y})}{\sqrt{\sum_{t=1}^{T}(y_t-\bar{y})^2\sum_{t=1}^{T}(y_{t+k}-\bar{y})^2}}
$$

其中，$\rho_k$表示滞后k阶的自相关系数，$T$表示时间序列的长度，$y_t$表示时间序列的第t个观测值，$\bar{y}$表示时间序列的均值。

In [ ]:
# 时间序列的趋势图
fig = plt.figure(figsize=(8,4))
sns.lineplot(x='year', y='y', data=data)

In [ ]:
# 对数时间序列的趋势图
fig = plt.figure(figsize=(8,4))
sns.lineplot(x='year', y='lny', data=data)

In [ ]:
# 对比
fig = plt.figure(figsize=(10,4))
sns.lineplot(x='year', y='dlny', data=data)
sns.lineplot(x='year', y='dy', data=data)

In [ ]:
from cq import acfgram

ac = acfgram(data['dlny'],lags=15)
ac


## 13.2 一阶自回归



In [ ]:
# 使用OLS回归
data2012 = data[data.index<2013].dropna()
mod = sm.OLS(endog=data2012['dlny'], 
             exog=sm.add_constant(data2012['dlny'].shift(1)),
             missing='drop') # 缺失值直接drop
res = mod.fit(cov_type='HC1', use_t=True)
print(res.summary())

In [ ]:
# 使用MLE
from statsmodels.tsa.ar_model import AutoReg
mod_mle = AutoReg(data2012['dlny'], lags=1)
res_mle = mod_mle.fit(cov_type='HC1',use_t=True)
print(res_mle.summary())

In [ ]:
from math import exp
# 预测值
a = res.predict([1,data.loc[2013,'dlny']])
# OLS模型回归后的预测：
## - 参数是对应的 const + 其他自变量的值
b = res_mle.predict(len(data['lny']),len(data['lny']))
# AutoReg模型回归后的预测：
## - 参数是start、end

resid = exp(a + data.loc[2012,'lny']) - data.loc[2013,'y']
resid

## 13.3 高阶自回归



## 13.4 自回归分布滞后模型




## 13.5 误差修正模型



## 13.6 移动平均与ARMA模型



## 13.7 脉冲响应函数



## 13.8 向量自回归过程



## 13.9 VAR的脉冲响应函数



## 13.10 格兰杰因果检验



## 13.11 VAR的Stata命令及实例



## 13.12 时间趋势项



## 13.13 季节调整



## 13.14 日期数据的导入